In [43]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/jarlstephansson/Desktop/ID2223/Lab1-ID2223
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [44]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [45]:
from dataclasses import dataclass

@dataclass
class Place:
    url: str
    country: str
    city: str
    street: str
    csv: str = None
    lat: str = None   # optional
    lon: str = None   # optional

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()
today = datetime.date.today()

objs = []


for i in range(3):  # we have 3 locations 
    
    # This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
    AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
    location_str = secrets.get_secret(f"SENSOR_LOCATION_JSON{i}").value
    
    
    location = json.loads(location_str)
    
    country=location['country']
    city=location['city']
    street=location['street']
    aqicn_url=location['aqicn_url']
    latitude=location['latitude']
    longitude=location['longitude']
    temp_obj = Place(
        url=aqicn_url,
        country=country,
        city=city,
        street=street,
        lat=latitude,
        lon=longitude)
    objs.append(temp_obj)



2025-11-18 12:45:53,302 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-18 12:45:53,307 INFO: Initializing external client
2025-11-18 12:45:53,308 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-18 12:45:54,731 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1272012


### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [46]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [47]:
import requests
import pandas as pd
temps = []

for idx, obj in enumerate(objs):
    temp = util.get_pm25(obj.url, obj.country, obj.city, obj.street, today, AQICN_API_KEY)

    temps.append(temp)

aq_today_df = pd.concat(temps, ignore_index=True)

In [48]:
aq_today_df

,pm25,country,city,street,date,url
0,39.0,sweden,hedemora,gussarvsgatan,2025-11-18,https://api.waqi.info/feed/@12727
1,4.0,sweden,w 675,smedjebacken,2025-11-18,https://api.waqi.info/feed/A84085
2,5.0,sweden,W 911,falun,2025-11-18,https://api.waqi.info/feed/A558664


In [49]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     3 non-null      float32       
 1   country  3 non-null      object        
 2   city     3 non-null      object        
 3   street   3 non-null      object        
 4   date     3 non-null      datetime64[ns]
 5   url      3 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 264.0+ bytes


In [50]:

df_prev = air_quality_fg.read()
df_prev = df_prev.sort_values(["city", "date"])

last3 = (
    df_prev.groupby("city")["pm25"]
           .tail(3)
           .groupby(df_prev["city"])
           .apply(list)
           .to_dict()
)

def get_rolls(city):
    vals = last3[city]  
    return pd.Series({
        "rolling1": vals[-1],  # yesterday
        "rolling2": vals[-2],  # 2 days ago
        "rolling3": vals[-3],  # 3 days ago
    })

rolls = aq_today_df["city"].apply(get_rolls)
aq_today_df[["rolling1", "rolling2", "rolling3"]] = rolls


aq_today_df = aq_today_df.astype({
    "rolling1": "float32",
    "rolling2": "float32",
    "rolling3": "float32"
})

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.13s) 


In [51]:
aq_today_df

,pm25,country,city,street,date,url,rolling1,rolling2,rolling3
0,39.0,sweden,hedemora,gussarvsgatan,2025-11-18,https://api.waqi.info/feed/@12727,11.0,11.0,22.0
1,4.0,sweden,w 675,smedjebacken,2025-11-18,https://api.waqi.info/feed/A84085,2.0,2.4,2.2
2,5.0,sweden,W 911,falun,2025-11-18,https://api.waqi.info/feed/A558664,4.0,3.6,4.2


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [52]:
tempss = []
for idx, obj in enumerate(objs):    
    hourly_df = util.get_hourly_weather_forecast(obj.city, obj.lat, obj.lon)
    hourly_df = hourly_df.set_index('date')

    # We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
    # We only want the daily weather data, so only get weather at 12:00
    daily_df = hourly_df.between_time('11:59', '12:01')
    daily_df = daily_df.reset_index()
    daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
    daily_df['date'] = pd.to_datetime(daily_df['date'])
    daily_df['city'] = obj.city
    tempss.append(daily_df)

daily_df = pd.concat(tempss, ignore_index=True)

Coordinates 60.25°N 16.0°E
Elevation 109.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 60.0°N 15.75°E
Elevation 150.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 60.5°N 15.5°E
Elevation 209.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [53]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         21 non-null     datetime64[ns]
 1   temperature_2m_mean          21 non-null     float32       
 2   precipitation_sum            21 non-null     float32       
 3   wind_speed_10m_max           21 non-null     float32       
 4   wind_direction_10m_dominant  21 non-null     float32       
 5   city                         21 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 804.0+ bytes


In [54]:
daily_df

,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-18,-1.00,0.0,9.387651,327.528839,hedemora
1,2025-11-19,-2.55,0.0,5.154415,24.775122,hedemora
2,2025-11-20,-2.85,0.0,14.973576,350.311279,hedemora
3,2025-11-21,-3.30,0.0,10.787993,295.709900,hedemora
4,2025-11-22,0.60,0.0,10.823973,266.186005,hedemora
5,2025-11-23,0.70,0.1,8.913181,133.363464,hedemora
6,2025-11-24,0.70,0.0,5.483356,336.801392,hedemora
7,2025-11-18,-1.25,0.0,5.860375,312.510406,w 675
8,2025-11-19,-2.35,0.0,6.109403,44.999897,w 675
9,2025-11-20,-3.40,0.0,13.138765,350.537750,w 675


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [55]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-18 12:46:04,400 INFO: 	4 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1272012/fs/1265759/fg/1721954


Uploading Dataframe: 100.00% |██████████| Rows 3/3 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1272012/jobs/named/air_quality_1_offline_fg_materialization/executions


(Job('air_quality_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "rolling1",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 762141
         }
       },
       "result": {
         "observed_value": 2.0,
         "element_count": 3,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T11:46:04.000400Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to

In [56]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-18 12:46:17,596 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1272012/fs/1265759/fg/1721955


Uploading Dataframe: 100.00% |██████████| Rows 21/21 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1272012/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-18 12:46:34,940 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-18 12:46:38,125 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 12:48:48,051 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2025-11-18 12:48:54,380 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-18 12:48:54,564 INFO: Waiting for log aggregation to finish.
2025-11-18 12:49:09,668 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 762144
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 21,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T11:46:17.000595Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
